<a href="https://colab.research.google.com/github/bubblepuzzle/ProductSummarization/blob/main/Major_Project_FINAL_DRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATASET EXPLORATION

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
import random
from google.colab import drive
from google.colab import files
import shutil

! pip install split-folders
! pip install -q kaggle

In [ ]:
!gdown 1CnfesdtttDI8BuE-Mxpg5c3LZETdpu2b
import zipfile as zf


Downloading...
From (original): https://drive.google.com/uc?id=1CnfesdtttDI8BuE-Mxpg5c3LZETdpu2b
From (redirected): https://drive.google.com/uc?id=1CnfesdtttDI8BuE-Mxpg5c3LZETdpu2b&confirm=t&uuid=d3cb70cd-89f7-434e-964b-fe98bdc202e1
To: /content/amazon_reviews_us_Baby_v1_00.tsv.zip
100% 362M/362M [00:03<00:00, 96.8MB/s]


In [ ]:
hehe=zf.ZipFile("amazon_reviews_us_Baby_v1_00.tsv.zip",'r')
hehe.extractall()

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
root_path = "/content/drive/MyDrive/REVIEW Major Proj"
dataset_dir = os.path.join(root_path, "DataSet")
dataset_path= os.path.join(dataset_dir, "amazon_reviews_us_Baby_v1_00.tsv.zip")

In [ ]:
#Decompress and load the tsv in a pandas data frame
reviews = pd.read_csv('/content/amazon_reviews_us_Baby_v1_00.tsv.zip', compression="zip", sep="\t", header=0, error_bad_lines=False)


In [ ]:
reviews.info()

In [ ]:
sample = reviews.head(200000)

In [ ]:
product_review_counts = reviews['product_id'].value_counts()

# Get the product IDs for the top 5 products with the highest review counts
top_products = product_review_counts.head(5).index.tolist()

# Create a subset of the dataset containing reviews for the top products
subset_reviews = reviews[reviews['product_id'].isin(top_products)]

In [ ]:
subset_reviews.info()

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
subset_reviews['review_year'] = pd.to_datetime(subset_reviews['review_date']).dt.year

# Display the updated DataFrame
print(subset_reviews.head())

In [ ]:
subset_reviews.info()

In [ ]:
subset_reviews.to_csv('/content/Reduced DataSet.csv', index=False)

We're using a subset of the starting dataset for simplicity, in particular it formed by the reviews of the 5 products with the highest number of reviews.

In [ ]:
sample.info()

In [ ]:
#Save the sample into a csv on the shared drive
sample.to_csv('/content/sample.csv', index=False)

DATA Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import string
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import drive


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
stopwords_en = stopwords.words('english')

TEXT PREPROCESSING

In the following we've defined a function that applies some text pre processing techniques:

Transform to lowercase

Remove the HTML tags

Tokenization

Stopwords removal

Remove the punctuation marks

In [ ]:
#Creation of a list of reviews, each review will be tokenized considering a word
# as a token.
def preprocess_review(review_list):

  #regex to remove HTML tags
  html_removal = re.compile('<.*?>')

  cleaned_reviews = []
  for review in review_list:
      #each review must be converted to lowercase
      lower_review = review.lower()

      #remove the html tags
      nohtml_review = re.sub(html_removal, ' ', lower_review)

      #tokenized the review using the word_tonkenize
      tokenized_review = word_tokenize(nohtml_review)

      #stopwords removal
      nostopwords_review = []

      #for each token, check if the token is inside the list of stopwords
      # for token in tokenized_review:
      #     if token not in stopwords_en:
      #         #append the ones not in the list of stopwords
      #         nostopwords_review.append(token)
      nostopwords_review = [token for token in tokenized_review if token not in stopwords_en]

      #remove punctuation #1
      #remove a token only if it is composed by a single punctuation mark
      #nopunctuation_review = [token for token in nostopwords_review if token not in string.punctuation]

      #remove punctuation #2
      #remove tokens composed entirely by punctuation marks
      nopunctuation_review = [token for token in nostopwords_review if not all(char in string.punctuation for char in token)]

      #remove punctuation #3 #we don't need to use tagging for punctuation!
      # pos_tagging = nltk.pos_tag(nostopwords_review)
      # nopunctuation_review = [tuple for tuple in pos_tagging if len(tuple[1]) > 1 ]

      cleaned_reviews.append(' '.join(nopunctuation_review))

  return cleaned_reviews

In [ ]:
#Function to write the list into a file
def write_list_to_file(filename, list):
  #Opening a file in 'w' mode clears its content
  file = open(filename,'w')
  for item in list:
    file.write(item+"\n")
  file.close()

In [ ]:
#Function to read a list from a file
def read_list_from_file(filename):
  result = []
  # opening the file in read mode
  my_file = open(filename, "r")

  # reading the file
  data = my_file.read()

  # replacing end splitting the text
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  my_file.close()

  return data_into_list

In [ ]:
#Mount Google Drive to access the shared folder
drive.mount('/content/drive')

In [ ]:
#Load the dataset as csv, skipping the bad lines
csv = pd.read_csv("/content/Reduced DataSet.csv", on_bad_lines="skip")

#Drop the null values
csv.dropna(inplace=True)

#Check the number of the reviews for each product in the reduced dataset
csv_grouped = csv.groupby(by="product_id").count().sort_values(by="review_body")
csv_grouped

In [ ]:
ax = csv_grouped.plot.bar(y='review_id', rot=0)

Dataset creation

For the test of the models we'll use just the reviews of the product with product_id B000YDDF6O, the features that we use are just the review_body and review_headline, the latter is used as label during the fine tuning process.

In [ ]:
csv_r = csv[csv["product_id"] == "B000YDDF6O"]
csv_r = csv_r[["review_body", "review_headline"]]
csv_r.reset_index(inplace=True, drop=True)

In [ ]:
csv_r.to_csv('/content/amazon_reviews_reduced_preprocessed.csv', index=False)


In [ ]:
#TEST
csv_r = pd.read_csv("/content/amazon_reviews_reduced_preprocessed.csv", on_bad_lines="skip")
#OK

In [ ]:
#Sample the reviews to be summarized
review_list = csv_r["review_body"].tail(1000).tolist()

In [ ]:
#Preprocess the list
review_list = preprocess_review(review_list)

In [ ]:
#Filename for the review list's file
filename_review_list = "/content/review_list.txt"

In [ ]:
#Save the list to the disk
write_list_to_file(filename_review_list, review_list)

In [ ]:
#Check if the list was correctly written
res = read_list_from_file(filename_review_list)

In [ ]:
#Check if thw two lists are the same list
all(x == y for x, y in zip(review_list, res))

Analyzing the reviews composition

In [ ]:
#Dict containing the number of tokens as key and the number of reviews having
# such a number of tokens
lengths = dict()
for review in review_list:
  length = len(review.split(" "))
  if length in lengths:
    lengths[length] += 1
  else:
    lengths[length] = 1

#Sort in increasing number of keys
sorted_lengths = dict(sorted(lengths.items()))

In [ ]:
plt.xlabel ('Number of tokens')
plt.ylabel ('Number of reviews')
plt.bar(sorted_lengths.keys(), sorted_lengths.values(), color='b')

In [ ]:
print("(number of tokens: {}, number of reviews: {})".format(max(lengths, key= lambda x: lengths[x]), lengths[max(lengths, key= lambda x: lengths[x])]))

In [ ]:
total = 0
#Sum the lengths of the reviews
for review in review_list:
  total += len(review.split(" "))

#Divide by the total number of reviews
avg = total / len(review_list)

print("Avg length of review: {}".format( avg) )

**T5 Test**

In [ ]:
!pip install -q tf-models-official
!pip install -q transformers
!pip install -q spacy
!pip install -q datasets
#!pip install -q pyarrow

In [ ]:
import tensorflow as tf
#import tensorflow_models as tfm
#import tensorflow_hub as hub
#import tensorflow_datasets as tfds
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
import datasets
import os
from numba import cuda


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")
stopwords_en = stopwords.words('english')

In [ ]:
#from huggingface_hub import notebook_login

#notebook_login()

In [ ]:
csv = pd.read_csv("drive/MyDrive/REVIEW Major Proj/DataSet/Reduced DataSet.csv", on_bad_lines="skip")
csv.dropna(inplace=True)
csv.groupby(by="product_id").count().sort_values(by="review_body")

In [ ]:
csv_r = csv[csv["product_id"] == "B000YDDF6O"]
csv_r = csv_r[["review_body", "review_headline"]]
csv_r.reset_index(inplace=True, drop=True)
review_headline = csv_r["review_headline"].tail(1000).tolist()
review_list = csv_r["review_body"].tail(1000).tolist()

In [ ]:
review_list

In [ ]:
dataset = ds.dataset(pa.Table.from_pandas(csv_r).to_batches())
dataset = Dataset(pa.Table.from_pandas(csv_r[:2000]))
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
p = dataset.get("train")
max_l = 0
for elem in p:
  max_l = max_l + len(elem["review_body"])
max_l/1000.0

In [ ]:
prefix = "summarize: "
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["review_body"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["review_headline"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
# SPLIT DATASET
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-base", return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

In [ ]:
#from transformers.keras_callbacks import PushToHubCallback
#model.compile(optimizer=optimizer)

#model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)


In [ ]:
import re
import string
#Creation of a list of reviews, each review will be tokenized considering a word
# as a token.

print(review_list[0])

#regex to remove HTML tags
html_removal = re.compile('<.*?>')

cleaned_reviews = []
for review in review_list:
    #each review must be converted to lowercase
    lower_review = review.lower()

    #remove the html tags
    nohtml_review = re.sub(html_removal, ' ', lower_review)

    #tokenized the review using the word_tonkenize
    tokenized_review = word_tokenize(nohtml_review)

    #stopwords removal
    nostopwords_review = []

    #for each token, check if the token is inside the list of stopwords
    # for token in tokenized_review:
    #     if token not in stopwords_en:
    #         #append the ones not in the list of stopwords
    #         nostopwords_review.append(token)
    nostopwords_review = [token for token in tokenized_review if token not in stopwords_en]

    #remove punctuation #1
    #remove a token only if it is composed by a single punctuation mark
    #nopunctuation_review = [token for token in nostopwords_review if token not in string.punctuation]

    #remove punctuation #2
    #remove tokens composed entirely by punctuation marks
    nopunctuation_review = [token for token in nostopwords_review if not all(char in string.punctuation for char in token)]

    #remove punctuation #3 #we don't need to use tagging for punctuation!
    # pos_tagging = nltk.pos_tag(nostopwords_review)
    # nopunctuation_review = [tuple for tuple in pos_tagging if len(tuple[1]) > 1 ]

    cleaned_reviews.append(' '.join(nopunctuation_review))

print(cleaned_reviews[0])

In [ ]:
# reviews = []
# for i in range(0, 400):
#     sequence = cleaned_reviews[i]
#     inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
#     output = model.generate(inputs, max_length=100)
#     summary=tokenizer.decode(output[0])
#     reviews.append(summary)
# seq = ' '.join(reviews)
# inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
# output = model.generate(inputs, max_length=100)
# summary=tokenizer.decode(output[0])
# print("T5 summary nltk: " + summary)
input = "summarize: "
count = 0
checkpoint = 0


#batch creation


batches = []
i = 0
for i in range(0,100):
  if count + len(cleaned_reviews[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + cleaned_reviews[i] + " . "
    #print(input)
    count = count + len(cleaned_reviews[i].split(" "))
  else:
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + cleaned_reviews[i] + " . "
    count = len(cleaned_reviews[i].split(" "))

if(count != 0):
  batches.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches), count))
  print("Number of reviews processed: {}".format(i))



In [ ]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_first_stage

In [ ]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

In [ ]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_second_stage

In [ ]:
reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("T5 summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = cleaned_reviews[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("T5 summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("T5 summary spacy: " + summary)

#device = cuda.get_current_device()
#device.reset()

In [ ]:
reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("bart-base summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("bart-base summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model.generate(inputs, max_length=100)
    summary=tokenizer.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model.generate(inputs, max_length=100)
summary=tokenizer.decode(output[0])
print("bart-base summary spacy: " + summary)

device = cuda.get_current_device()
device.reset()


**STOPWORD ELIMINATION WITH NLTK AND TEXT REDUCTION**

In [ ]:
nltk_test = []
for elem in review_list:
    nltk_test.append(word_tokenize(elem))

nltk_un = []
for text in nltk_test:
    temp = []
    for w in text:
        if w not in stop_words:
            temp.append(w)
    nltk_un.append(' '.join(temp))

nltk_fin = []
for elem in nltk_un:
    if len(elem) <= 1000:
        nltk_fin.append(elem)

rev = []
for doc in review_list:
    words = []
    for token in nlp(doc):
        if (token.pos_ not in ['VERB', 'ADJ', 'NOUN', 'DET', 'PUNCT']):
        #if (token.pos_ == 'VERB' or token.pos_ == 'ADJ' or token.pos_ == 'NOUN' or token.pos_ == 'DET' or token.pos_ == 'PUNCT'):
            words.append(token.lemma_)
    rev.append(words)

fin = []
for doc in rev:
    fin.append(" ".join(doc))

**T5 TEST WITH FULL TEXT, NLTK**

In [ ]:
tokenizer_t5 = AutoTokenizer.from_pretrained('t5-base')
model_t5 = TFAutoModelForSeq2SeqLM.from_pretrained('t5-base', return_dict=True)

print()

reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer_t5.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_t5.generate(inputs, max_length=100)
    summary=tokenizer_t5.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_t5.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_t5.generate(inputs, max_length=100)
summary=tokenizer_t5.decode(output[0])
print("T5 summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer_t5.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_t5.generate(inputs, max_length=100)
    summary=tokenizer_t5.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_t5.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_t5.generate(inputs, max_length=100)
summary=tokenizer_t5.decode(output[0])
print("T5 summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer_t5.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_t5.generate(inputs, max_length=100)
    summary=tokenizer_t5.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_t5.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_t5.generate(inputs, max_length=100)
summary=tokenizer_t5.decode(output[0])
print("T5 summary spacy: " + summary)

**BART TEST WITH FULL TEXT**

In [ ]:
tokenizer_bart = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model_bart = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn', return_dict=True)

print()

reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer_bart.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_bart.generate(inputs, max_length=100)
    summary=tokenizer_bart.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_bart.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_bart.generate(inputs, max_length=100)
summary=tokenizer_bart.decode(output[0])
print("Bart summary total: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer_bart.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_bart.generate(inputs, max_length=100)
    summary=tokenizer_bart.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_bart.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_bart.generate(inputs, max_length=100)
summary=tokenizer_bart.decode(output[0])
print("Bart summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer_bart.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=1024)
    output = model_bart.generate(inputs, max_length=100)
    summary=tokenizer_bart.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_bart.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=1024)
output = model_bart.generate(inputs, max_length=100)
summary=tokenizer_bart.decode(output[0])
print("Bart summary spacy: " + summary)

**PEGASUS TEST WITH FULL TEXT**

In [ ]:
tokenizer_pega = AutoTokenizer.from_pretrained('google/pegasus-xsum')
model_pega = TFAutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum', return_dict=True)

print()

reviews = []
for i in range(0, 50):
    sequence = review_list[i]
    inputs  = tokenizer_pega.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=512)
    output = model_pega.generate(inputs, max_length=100)
    summary=tokenizer_pega.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_pega.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=512)
output = model_pega.generate(inputs, max_length=100)
summary=tokenizer_pega.decode(output[0])
print("Pegasus summary totale: " + summary)

reviews = []
for i in range(0, 50):
    sequence = nltk_fin[i]
    inputs  = tokenizer_pega.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=512)
    output = model_pega.generate(inputs, max_length=100)
    summary=tokenizer_pega.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_pega.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=512)
output = model_pega.generate(inputs, max_length=100)
summary=tokenizer_pega.decode(output[0])
print("Pegasus summary nltk: " + summary)

reviews = []
for i in range(0, 50):
    sequence = fin[i]
    inputs  = tokenizer_pega.encode("summarize: " + sequence, return_tensors='tf', truncation=True, max_length=512)
    output = model_pega.generate(inputs, max_length=100)
    summary=tokenizer_pega.decode(output[0])
    reviews.append(summary)
seq = ' '.join(reviews)
inputs  = tokenizer_pega.encode("summarize: " + seq, return_tensors='tf', truncation=True, max_length=512)
output = model_pega.generate(inputs, max_length=100)
summary=tokenizer_pega.decode(output[0])
print("Pegasus summary spacy: " + summary)

**T5 BASE FINETUNING**

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

In [ ]:
#Function to read a list from a file
def read_list_from_file(filename):
  result = []
  # opening the file in read mode
  my_file = open(filename, "r")

  # reading the file
  data = my_file.read()

  # replacing end splitting the text
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  my_file.close()

  return data_into_list

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Filename of the review list's file
filename_review_list = "/content/review_list.txt"

#Filename of the dataset
filename_dataset = "/content/Reduced DataSet.csv"

In [ ]:
amazon_reviews = pd.read_csv(filename_dataset, on_bad_lines="skip")

In [ ]:
amazon_reviews.info()

DEFINITION OF FUNCTIONS

In [ ]:
def get_dataset_from_product_id(df, product_id, number_of_samples, test_size, shuffle):
  csv_r = df[df["product_id"] == product_id]
  csv_r = csv_r[["review_body", "review_headline"]]
  csv_r = csv_r[csv_r["review_headline"] != "Five Stars"]

  #TODO
  # review_body and review_headline must be preprocessed

  dataset = Dataset.from_pandas(df = csv_r.sample(number_of_samples))
  dataset = dataset.train_test_split(test_size = test_size)

  return dataset

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    # inputs = [prefix + doc for doc in examples["review_body"]]
    inputs = [doc for doc in examples["review_body"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["review_headline"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def plot_training_history(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  i = np.argmin(val_loss)
  x_min = epochs[i]
  y_min = val_loss[i]
  plt.plot(x_min, y_min,'g',marker='o', label="Minimum validation loss")

  plt.plot(epochs, loss, 'r', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))

  return batches

**TEST TRAINING 1000**

In [ ]:
dataset = get_dataset_from_product_id(amazon_reviews, "B000YDDF6O", 1000, 0.2, True)
print(dataset)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(tokenized_dataset)

In [ ]:
# SPLIT DATASET
optimizer = AdamWeightDecay(learning_rate=1e-3, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained('t5-base')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-base", return_tensors="tf")

In [ ]:
# SPLIT DATASET

#We don't need to separate the dataset in training and test manually
# prepare_tf_dataset split automatically
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator
)

In [ ]:
model.compile(optimizer=optimizer)
history = model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

In [ ]:
plot_training_history(history)

**PREPROCESS REVIEWS**

In [ ]:
cleaned_reviews = read_list_from_file(filename_review_list)
print(cleaned_reviews[0])

In [ ]:
len(cleaned_reviews)

In [ ]:
reviews_list_preprocessed = read_list_from_file(filename_review_list)
batches = get_batches(reviews_list_preprocessed, True)

In [ ]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  # print(decoded_output)
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_first_stage

In [ ]:
batches[0]

In [ ]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

In [ ]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_second_stage

**T5 base finetuning large**

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

In [ ]:
amazon_reviews = pd.read_csv(filename_dataset, on_bad_lines="skip")

In [ ]:
amazon_reviews.info()

In [ ]:
model.compile(optimizer=optimizer)
history = model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

In [ ]:
plot_training_history(history)

**test training 4000**

In [ ]:
import pandas as pd
import numpy as np

def get_dataset_from_product_id(df, product_id, num_samples, test_size, random_state):
    product_reviews = df[df['product_id'] == product_id]
    if len(product_reviews) == 0:
        raise ValueError("No reviews found for product_id: {}".format(product_id))

    obj_len = len(product_reviews)
    if obj_len == 0:
        raise ValueError("No reviews found for product_id: {}".format(product_id))

    train_size = int(obj_len * (1 - test_size))
    train_indices = random_state.choice(obj_len, size=train_size, replace=False)
    test_indices = np.setdiff1d(np.arange(obj_len), train_indices)

    train_set = product_reviews.iloc[train_indices]
    test_set = product_reviews.iloc[test_indices]

    return train_set, test_set

# Example usage
amazon_reviews = pd.read_csv(filename_dataset, on_bad_lines="skip")
random_state = np.random.default_rng(42)
train_set, test_set = get_dataset_from_product_id(amazon_reviews, "B000YDDF6O", 4000, 0.2, random_state)
print(train_set)
print(test_set)


In [ ]:
cleaned_reviews = read_list_from_file(filename_review_list)
print(cleaned_reviews[0])

In [ ]:
len(cleaned_reviews)

In [ ]:
reviews_list_preprocessed = read_list_from_file(filename_review_list)
batches = get_batches(reviews_list_preprocessed, True)

In [ ]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  # print(decoded_output)
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_first_stage

In [ ]:
batches[0]

In [ ]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

In [ ]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_second_stage

**T5 Large finetuning**

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

In [ ]:
#Filename of the review list's file
filename_review_list = "/content/review_list.txt"

#Filename of the dataset
filename_dataset = "/content/Reduced DataSet.csv"

In [ ]:
amazon_reviews = pd.read_csv(filename_dataset, on_bad_lines="skip")

In [ ]:
amazon_reviews.info()

In [ ]:
def get_dataset_from_product_id(df, product_id, number_of_samples, test_size, shuffle):
  csv_r = df[df["product_id"] == product_id]
  csv_r = csv_r[["review_body", "review_headline"]]
  csv_r = csv_r[csv_r["review_headline"] != "Five Stars"]

  #TODO
  # review_body and review_headline must be preprocessed

  dataset = Dataset.from_pandas(df = csv_r.sample(number_of_samples))
  dataset = dataset.train_test_split(test_size = test_size)

  return dataset

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    # inputs = [prefix + doc for doc in examples["review_body"]]
    inputs = [doc for doc in examples["review_body"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["review_headline"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def plot_training_history(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  i = np.argmin(val_loss)
  x_min = epochs[i]
  y_min = val_loss[i]
  plt.plot(x_min, y_min,'g',marker='o', label="Minimum validation loss")

  plt.plot(epochs, loss, 'r', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation
  #creo un array di testi concatenati, ognuno dei quali ha massimo 1024 token
  #lo stesso procedimento verrà fatto con i riassunti generati!
  # si parte da checkpoint in poi, fino a finire il ciclo.

  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))

  return batches

T5 LARGE Training 1000

In [ ]:
dataset = get_dataset_from_product_id(amazon_reviews, "B000YDDF6O", 1000, 0.2, True)
print(dataset)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-large')

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(tokenized_dataset)
#Maybe an error!! We've review_body, review headline, input_ids, ... but we need just input ids and labels!

In [ ]:
# SPLIT DATASET
optimizer = AdamWeightDecay(learning_rate=1e-3, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained('t5-large')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-large", return_tensors="tf")

In [ ]:
# SPLIT DATASET

#We don't need to separate the dataset in training and test manually
# prepare_tf_dataset split automatically
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator
)

In [ ]:
model.compile(optimizer=optimizer)
history = model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

In [ ]:
plot_training_history(history)

**Final Data Set**

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import math
import string
import matplotlib.pyplot as plt

In [ ]:
#Load the dataset as csv, skipping the bad lines
csv = pd.read_csv("/content/Reduced DataSet.csv", on_bad_lines="skip")

#Drop the null values
csv.dropna(inplace=True)

#Check the number of the reviews for each product in the reduced dataset
csv_grouped = csv.groupby(by="product_id").count().sort_values(by="review_body")
csv_grouped

In [ ]:
csv.head(2)

**DATASET CREATION**

In [ ]:
csv_r = csv[csv["product_id"] == "B000YDDF6O"]
csv_r = csv_r[["review_body", "review_headline", "star_rating","helpful_votes","review_date"]]
csv_r.reset_index(inplace=True, drop=True)
csv_r

In [ ]:
# csv_r["review_weight"] = math.log(csv_r["helpful_votes"]+1)*csv_r["star_rating"]

csv_r["review_weight"] = csv_r.apply(lambda x: math.log(x["helpful_votes"]+1)*x["star_rating"], axis=1)

In [ ]:
csv_r[csv_r["review_weight"] != 0]

In [ ]:
sum(csv_r["star_rating"])/csv_r['star_rating'].count()

In [ ]:

#weighted average based on helpful votes
sum(csv_r[csv_r["review_weight"] != 0]["review_weight"])/sum(csv_r[csv_r["review_weight"] != 0].apply(lambda x: math.log(x['helpful_votes']+1), axis=1))

In [ ]:
sum(csv_r[csv_r["review_weight"] != 0]["review_weight"])

In [ ]:
sum(csv_r[csv_r["review_weight"] != 0].apply(lambda x: math.log(x['helpful_votes']+1), axis=1))

In [ ]:

#weighted average based on helpful votes
sum(csv_r['star_rating']*(csv_r['helpful_votes']+0.1))/sum(csv_r['helpful_votes']+0.1) #Adding significance to the ones with 0

In [ ]:

#weighted average based on helpful votes
sum(csv_r['star_rating']*(csv_r['helpful_votes']))/sum(csv_r['helpful_votes']) #Without significance

In [ ]:

#weighted average based on helpful votes logarithmic scale considering only those with weight != 0
sum(csv_r[csv_r["review_weight"] != 0]["review_weight"])/sum(csv_r[csv_r["review_weight"] != 0].apply(lambda x: math.log(x['helpful_votes']+1), axis=1))

In [ ]:
csv_r

In [ ]:
#Save the sample into a CSV file
csv_r.to_csv('/content/sample_weighted.csv', index=False)

BART FineTuning

In [ ]:
!pip install -q transformers
!pip install -q datasets

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os

import nltk
nltk.download('punkt')

In [ ]:
#Function to read a list from a file
def read_list_from_file(filename):
  result = []
  # opening the file in read mode
  my_file = open(filename, "r")

  # reading the file
  data = my_file.read()

  # replacing end splitting the text
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  my_file.close()

  return data_into_list

In [ ]:
#Filename of the review list's file
filename_review_list = "/content/review_list.txt"

#Filename of the dataset
filename_dataset = "/content/Reduced DataSet.csv"

In [ ]:
amazon_reviews = pd.read_csv("/content/Reduced DataSet.csv", on_bad_lines="skip")

In [ ]:
amazon_reviews.info()

In [ ]:
def get_dataset_from_product_id(df, product_id, number_of_samples, test_size, shuffle):
  csv_r = df[df["product_id"] == product_id]
  csv_r = csv_r[["review_body", "review_headline"]]

  #TODO
  # review_body and review_headline must be preprocessed

  dataset = Dataset.from_pandas(df = csv_r[:number_of_samples])
  dataset = dataset.train_test_split(test_size = test_size)

  return dataset

In [ ]:
dataset = get_dataset_from_product_id(amazon_reviews, "B000YDDF6O", 1000, 0.2, True)
print(dataset)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    # inputs = [prefix + doc for doc in examples["review_body"]]
    inputs = [doc for doc in examples["review_body"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["review_headline"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(tokenized_dataset)


In [ ]:
# SPLIT DATASET
optimizer = AdamWeightDecay(learning_rate=1e-3, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-base", return_tensors="tf")

In [ ]:
# SPLIT DATASET

#We don't need to separate the dataset in training and test manually
# prepare_tf_dataset split automatically
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator
)

In [ ]:
model.compile(optimizer=optimizer)

history = model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

In [ ]:
def plot_training_history(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  i = np.argmin(val_loss)
  x_min = epochs[i]
  y_min = val_loss[i]
  plt.plot(x_min, y_min,'g',marker='o', label="Minimum validation loss")

  plt.plot(epochs, loss, 'r', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
plot_training_history(history)

In [ ]:


cleaned_reviews = read_list_from_file(filename_review_list)
print(cleaned_reviews[0])

In [ ]:
input = ""
count = 0
checkpoint = 0


#batch creation

batches = []
i = 0
for i in range(0,43):
  if count + len(cleaned_reviews[i].split(" ")) < 800:
    #print(cleaned_reviews[i])
    input = input + cleaned_reviews[i] + " "
    #print(input)
    count = count + len(cleaned_reviews[i].split(" "))
  else:
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))
    input = "" + cleaned_reviews[i] + " . "
    count = len(cleaned_reviews[i].split(" "))

if(count != 0):
  batches.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches), count))
  print("Number of reviews processed: {}".format(i))

In [ ]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  print(decoded_output)
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_first_stage

In [ ]:
batches[0]

In [ ]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

In [ ]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_second_stage

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-xsum", tokenizer="facebook/bart-large-xsum")

In [ ]:
text = batches[0]
summary = summarizer( text, max_length=1024, min_length=10, do_sample=False)
print(summary)

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

**BART LARGE XSUM**

Second test: Introduction
In the second test with BART we'll use a more complex model trained bart-large-xsum specialized for tasks of summarization.

As before we use the following alternative inputs:

Preprocessed reviews
Full text
We'll leverage on the huggingface's transformers library also in this case.

In [ ]:
!pip install -q transformers

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from transformers import pipeline

import nltk
nltk.download('punkt')

In [ ]:
#Function to read a list from a file
def read_list_from_file(filename):
  result = []
  # opening the file in read mode

  my_file = open(filename, "r")

  # reading the file
  data = my_file.read()

  # replacing end splitting the text
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  my_file.close()

  return data_into_list

In [ ]:
#Filename of the review list's file
filename_review_list = "/content/review_list.txt"

#Filename of the dataset
filename_dataset = "/content/Reduced DataSet.csv"

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-xsum", tokenizer="facebook/bart-large-xsum")

In [ ]:
#These function receive as input a batch of reviews and for each one it outputs
# a number of summaries equal to the number of batches
def get_summaries(batches):
  summaries = []
  for input in batches:
    summary = summarizer( input, max_length=1024, min_length=10, do_sample=False)
    summaries.append(summary)
  return summaries

Creating the inputs

In the following cell we can see how the inputs are created; in particular we build batches of reviews, each one formed by the concatenation of different reviews.

In [ ]:
def get_batches(reviews, sample):

  input = " "
  count = 0
  checkpoint = 0


  #batch creation


  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < (512):
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      print("Number of reviews processed: {}".format(i))
      input = reviews[i] + " "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))

  return batches

Now we create the batches for the summaries

The previous batches of reviews form the input to the first stage of summarization: the results will be composed by N summaries, where N is the number of inputs to the model (the size of batches)

Loading the preprocessed reviews

In [ ]:

reviews_list_preprocessed = read_list_from_file(filename_review_list)
print(reviews_list_preprocessed[0])

In [ ]:
batches = get_batches(reviews_list_preprocessed, True)

In [ ]:
amazon_reviews = pd.read_csv("/content/amazon_reviews_reduced_preprocessed.csv", on_bad_lines="skip")

In [ ]:
review_list = amazon_reviews["review_body"].tail(1000).tolist()

In [ ]:
batches_full_text = get_batches(review_list, True)

Using the preprocessed reviews

The previous batches of reviews form the input to the first stage of summarization: the results will be composed by N summaries, where N is the number of inputs to the model (the size of batches)

In [ ]:
summaries_first_stage = get_summaries(batches=batches)

In [ ]:
len(batches[0].split(" "))

In [ ]:
summaries_first_stage = [x[0]["summary_text"] for x in summaries_first_stage]
summaries_first_stage


In [ ]:
print(summaries_first_stage)
print(batches)

In [ ]:
batches_second_stage = get_batches(summaries_first_stage, False)

In [ ]:
summaries_second_stage = get_summaries(batches_second_stage)

In [ ]:
print(summaries_second_stage[0])
print(summaries_first_stage[0])

In [ ]:
#Sentiment analysis
#Bert-base-multilingual-uncased-sentiment is a model fine-tuned for sentiment analysis on product reviews in six languages: English, Dutch, German, French, Spanish and Italian.
pipe = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
#replace the specific product

review_stars = pd.read_csv("/content/Reduced DataSet.csv", on_bad_lines="skip")
mask = review_stars[review_stars["product_id"] == "B000YDDF6O"]
mask = mask["star_rating"]

In [ ]:
val = pipe(summaries_first_stage[0])
print("First Stage: " + str(val[0].get("score")))

val = pipe(summaries_second_stage[0][0].get("summary_text"))
print("Second Stage: " + str(val[0].get("score")))

print("Expected Rating: " + str((mask.sum()/len(mask))/5))

Using the full text without preprocessing


The second part of the test on BART-base consists in the usage of the full text (without preprocessing) as input

In [ ]:
summaries_first_stage_full_text = get_summaries(batches=batches_full_text)

In [ ]:
summaries_first_stage_full_text = [x[0]["summary_text"] for x in summaries_first_stage_full_text]
summaries_first_stage_full_text


In [ ]:
batches_second_stage_full_text = get_batches(summaries_first_stage_full_text, False)

In [ ]:
summaries_second_stage_full_text = get_summaries(batches_second_stage_full_text)

In [ ]:
summaries_second_stage_full_text = [x[0]["summary_text"] for x in summaries_second_stage_full_text]
summaries_second_stage_full_text

In [ ]:
print(summaries_second_stage_full_text[0])
print(summaries_first_stage_full_text[0])
print(batches_full_text[0])

In [ ]:
#Sentiment analysis
#Bert-base-multilingual-uncased-sentiment is a model fine-tuned for sentiment analysis on product reviews in six languages: English, Dutch, German, French, Spanish and Italian.
pipe = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")


In [ ]:
review_stars = pd.read_csv("/content/Reduced DataSet.csv", on_bad_lines="skip")
mask = review_stars[review_stars["product_id"] == "B000YDDF6O"]
mask = mask["star_rating"]

In [ ]:
val = pipe(summaries_first_stage_full_text[0])
print("First Stage: " + str(val[0].get("score")))

val = pipe(summaries_second_stage_full_text)
print("Second Stage: " + str(val[0].get("score")))

print("Expected Rating: " + str((mask.sum()/len(mask))/5))

Third test: Introduction

In the second test with BART we'll use a more complex model trained bart-large-cnn specialized for tasks of summarization.

As before we use the following alternative inputs:

Preprocessed reviews
Full text
We'll leverage on the huggingface's transformers library also in this case.

In [ ]:
!pip install -q transformers

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from transformers import pipeline

import nltk
nltk.download('punkt')

In [ ]:
#Function to read a list from a file
def read_list_from_file(filename):
  result = []
  # opening the file in read mode
  my_file = open(filename, "r")

  # reading the file
  data = my_file.read()

  # replacing end splitting the text
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  my_file.close()

  return data_into_list

Defining the model

For this test we'll use bart-large-cnn

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
#These function receive as input a batch of reviews and for each one it outputs
# a number of summaries equal to the number of batches
def get_summaries(batches):
  summaries = []
  for input in batches:
    summary = summarizer( input, max_length=1024, min_length=10, do_sample=False)
    summaries.append(summary)
  return summaries

Creating the inputs

In the following cell we can see how the inputs are created; in particular we build batches of reviews, each one formed by the concatenation of different reviews.

In [ ]:
def get_batches(reviews, sample):

  input = " "
  count = 0
  checkpoint = 0


  #batch creation
  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < (512):
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      print("Number of reviews processed: {}".format(i))
      input = reviews[i] + " "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))

  return batches

Now we create the batches for the summaries


The previous batches of reviews form the input to the first stage of summarization: the results will be composed by N summaries, where N is the number of inputs to the model (the size of batches)

Loading the preprocessed reviews

In [ ]:


reviews_list_preprocessed = read_list_from_file(filename_review_list)
print(reviews_list_preprocessed[0])

In [ ]:
batches = get_batches(reviews_list_preprocessed, True)

In [ ]:
amazon_reviews = pd.read_csv("/content/amazon_reviews_reduced_preprocessed.csv", on_bad_lines="skip")

In [ ]:
review_list = amazon_reviews["review_body"].tail(1000).tolist()

In [ ]:
batches_full_text = get_batches(review_list, True)

Using the preprocessed reviews

The previous batches of reviews form the input to the first stage of summarization: the results will be composed by N summaries, where N is the number of inputs to the model (the size of batches)

In [ ]:
summaries_first_stage = get_summaries(batches=batches)

In [ ]:
len(batches[0].split(" "))

In [ ]:
summaries_first_stage = [x[0]["summary_text"] for x in summaries_first_stage]
summaries_first_stage


In [ ]:
print(summaries_first_stage)
print(batches)

In [ ]:
batches_second_stage = get_batches(summaries_first_stage, False)

In [ ]:
summaries_second_stage = get_summaries(batches_second_stage)

In [ ]:
print(summaries_second_stage[0])
print(summaries_first_stage[0])

Using the full text without preprocessing

The second part of the test on BART-base consists in the usage of the full text (without preprocessing) as input

In [ ]:
summaries_first_stage_full_text = get_summaries(batches=batches_full_text)

In [ ]:
summaries_first_stage_full_text = [x[0]["summary_text"] for x in summaries_first_stage_full_text]
summaries_first_stage_full_text


In [ ]:
batches_second_stage_full_text = get_batches(summaries_first_stage_full_text, False)

In [ ]:
summaries_second_stage_full_text = get_summaries(batches_second_stage_full_text)

In [ ]:
print(summaries_first_stage_full_text[0])
print(batches_full_text[0])

**PEGASUS BASE**

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

In [ ]:
#Function to read a list from a file
def read_list_from_file(filename):
  result = []
  # opening the file in read mode
  my_file = open(filename, "r")

  # reading the file
  data = my_file.read()

  # replacing end splitting the text
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  my_file.close()

  return data_into_list

In [ ]:
#Filename of the review list's file
filename_review_list = "/content/review_list.txt"

#Filename of the dataset
filename_dataset = "/content/Reduced DataSet.csv"

In [ ]:
amazon_reviews = pd.read_csv(filename_dataset, on_bad_lines="skip")

In [ ]:
amazon_reviews.info()

In [ ]:
def get_dataset_from_product_id(df, product_id, number_of_samples, test_size, shuffle):
  csv_r = df[df["product_id"] == product_id]
  csv_r = csv_r[["review_body", "review_headline"]]
  csv_r = csv_r[~csv_r["review_headline"].isin(["Five Stars", "Four Stars", "Three Stars", "Two Stars", "One Star"])]


  dataset = Dataset.from_pandas(df = csv_r.sample(number_of_samples))
  dataset = dataset.train_test_split(test_size = test_size)

  return dataset

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    # inputs = [prefix + doc for doc in examples["review_body"]]
    inputs = [doc for doc in examples["review_body"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["review_headline"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def plot_training_history(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  i = np.argmin(val_loss)
  x_min = epochs[i]
  y_min = val_loss[i]
  plt.plot(x_min, y_min,'g',marker='o', label="Minimum validation loss")

  plt.plot(epochs, loss, 'r', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation
  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))

  return batches

**TEST  Training**

In [ ]:
dataset = get_dataset_from_product_id(amazon_reviews, "B000YDDF6O", 4000, 0.2, True)
print(dataset)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(tokenized_dataset)
#Maybe an error!! We've review_body, review headline, input_ids, ... but we need just input ids and labels!

In [ ]:
# SPLIT DATASET
optimizer = AdamWeightDecay(learning_rate=1e-3, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google/pegasus-xsum", return_tensors="tf")

In [ ]:
# SPLIT DATASET


tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=2,
    collate_fn=data_collator
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=True,
    batch_size=2,
    collate_fn=data_collator
)

In [ ]:
model.compile(optimizer=optimizer)
history = model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

In [ ]:
plot_training_history(history)

In [ ]:
#load the preprocessed reviews BUT!! We've pretrained the network on the same ones, it's an error
# maybe we don't need the preprocessing or we can skip the first 1000 reviews

cleaned_reviews = read_list_from_file(filename_review_list)
print(cleaned_reviews[0])

In [ ]:
len(cleaned_reviews)

In [ ]:
reviews_list_preprocessed = read_list_from_file(filename_review_list)
batches = get_batches(reviews_list_preprocessed, True)

In [ ]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  # print(decoded_output)
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_first_stage

In [ ]:
batches[0]

In [ ]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

In [ ]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_second_stage

DATASET SPLIT BART

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

In [ ]:
filename_dataset = "/content/sample_weighted.csv"
df = pd.read_csv(filename_dataset)
df

In [ ]:
date = df['review_date']
date_list = []
for d in date:
  elem = d.split('-')
  date_list.append(elem[0])
date_list = [eval(i) for i in date_list]

In [ ]:
plt.hist(date_list, bins=12)


In [ ]:
df['review_date'] = date_list
df

In [ ]:
df_2010 = df[df['review_date'] == 2010]
df_2011 = df[df['review_date'] == 2011]
df_2012 = df[df['review_date'] == 2012]
df_2013 = df[df['review_date'] == 2013]
df_2014 = df[df['review_date'] == 2014]
df_2015 = df[df['review_date'] == 2015]

In [ ]:
df_2010

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-xsum')
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-xsum')

In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation

  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))

  return batches

In [ ]:
reviews = df_2010['review_body'].tolist()

In [ ]:
batches = get_batches(reviews, False)

In [ ]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  # print(decoded_output)
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_first_stage

In [ ]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

In [ ]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_second_stage

DATASET SPLIT PEGASUS

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

In [ ]:
filename_dataset = "/content/sample_weighted.csv"
df = pd.read_csv(filename_dataset)
df

In [ ]:
date = df['review_date']
date_list = []
for d in date:
  elem = d.split('-')
  date_list.append(elem[0])
date_list = [eval(i) for i in date_list]

In [ ]:
plt.hist(date_list, bins=12)

In [ ]:
df['review_date'] = date_list
df

In [ ]:
df_2010 = df[df['review_date'] == 2010]
df_2011 = df[df['review_date'] == 2011]
df_2012 = df[df['review_date'] == 2012]
df_2013 = df[df['review_date'] == 2013]
df_2014 = df[df['review_date'] == 2014]
df_2015 = df[df['review_date'] == 2015]

In [ ]:
df_2010

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')
model = TFAutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum')

In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation
  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    print("Number of reviews processed: {}".format(i))

  return batches

In [ ]:
reviews = df_2014['review_body'].tolist()

In [ ]:
batches = get_batches(reviews, False)

In [ ]:
summaries_first_stage = []
for input in batches:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  # print(decoded_output)
  summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
max_input_length = 512

summaries_first_stage = []
for input_text in batches:
    # Split the input into segments of max_input_length
    segments = [input_text[i:i + max_input_length] for i in range(0, len(input_text), max_input_length)]

    for segment in segments:
        inputs = tokenizer.encode(segment, return_tensors='tf', truncation=True, max_length=max_input_length)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])


In [ ]:
summaries_first_stage

In [ ]:
input = "summarize: "
count = 0
checkpoint = 0
batches_second_stage = []
i = 0
for i in range(0,len(summaries_first_stage)):
  if count + len(summaries_first_stage[i].split(" ")) < 1024:
    #print(cleaned_reviews[i])
    input = input + summaries_first_stage[i] + " . "
    #print(input)
    count = count + len(summaries_first_stage[i].split(" "))
  else:
    batches_second_stage.append(input)
    print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
    print("Number of reviews processed: {}".format(i))
    input = "summarize: " + summaries_first_stage[i] + " . "
    count = len(summaries_first_stage[i].split(" "))

if(count != 0):
  batches_second_stage.append(input)
  print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
  print("Number of reviews processed: {}".format(i))

print(batches_second_stage)

In [ ]:
summaries_second_stage = []
for input in batches_second_stage:
  inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
  output = model.generate(inputs, max_length=100)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

In [ ]:
summaries_second_stage

**FINAL TEST (0)**

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
df = pd.read_csv("/content/Reduced DataSet.csv")
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda
from transformers import pipeline

import nltk
nltk.download('punkt')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda
from transformers import pipeline

import nltk
nltk.download('punkt')

In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation


  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      #print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    #print("Number of reviews processed: {}".format(i))

  return batches

In [ ]:
def summarize_final(df):
  final_sum = []
  ids = df['product_id'].unique()
  years = df['review_date'].unique()
  for year in years:
      part = df[(df["product_id"] == "B000YDDF6O") & (df["review_year"] == year)]
      reviews = part["review_body"].tolist()
      batches = get_batches(reviews, False)

      summaries_first_stage = []
      for input in batches:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        # print(decoded_output)
        summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      input = "summarize: "
      count = 0
      checkpoint = 0
      batches_second_stage = []
      i = 0
      for i in range(0,len(summaries_first_stage)):
        if count + len(summaries_first_stage[i].split(" ")) < 1024:
          #print(cleaned_reviews[i])
          input = input + summaries_first_stage[i] + " . "
          #print(input)
          count = count + len(summaries_first_stage[i].split(" "))
        else:
          batches_second_stage.append(input)
          #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
          #print("Number of reviews processed: {}".format(i))
          input = "summarize: " + summaries_first_stage[i] + " . "
          count = len(summaries_first_stage[i].split(" "))

      if(count != 0):
        batches_second_stage.append(input)
        #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
        #print("Number of reviews processed: {}".format(i))

      summaries_second_stage = []
      for input in batches_second_stage:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      print("Product " + str(id) + ", year " + str(year) + ": " + str(summaries_second_stage))
      if(len(summaries_second_stage) != 0):
        final_sum.append(summaries_second_stage)
  return final_sum

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-xsum')
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-xsum')

In [ ]:
summaries = summarize_final(df)

In [ ]:
ds = df.groupby(by=['product_id', 'review_year'], sort=True).mean()
ds = ds.sort_values(by=['product_id', 'review_year'], ascending=[True, True])
ds = ds['star_rating'].tolist()
ds

In [ ]:
#Sentiment analysis
#Bert-base-multilingual-uncased-sentiment is a model fine-tuned for sentiment analysis on product reviews in six languages: English, Dutch, German, French, Spanish and Italian.
pipe = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
i = 0
for sum in summaries:
  val = pipe(str(sum))
  print("Second Stage: " + str(val[0].get("score")))
  print("Expected Rating: " + str(ds[i]/5))

**FINAL TEST**

In [ ]:
!pip install -q transformers datasets numba

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, TrainingArguments, Trainer, create_optimizer, AdamWeightDecay, DataCollatorForSeq2Seq
from transformers.keras_callbacks import PushToHubCallback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import datasets
from datasets import Dataset
import os
from numba import cuda

import nltk
nltk.download('punkt')

In [ ]:
df = pd.read_csv("/content/Reduced DataSet.csv")
df

In [ ]:
df.info()

In [ ]:
def get_batches(reviews, sample):

  input = "summarize:"
  count = 0
  checkpoint = 0


  #batch creation


  batches = []
  i = 0

  if sample:
    num_samples = 100
  else:
    num_samples = len(reviews)

  for i in range(0,num_samples):
  # for i in range(0,43):
    if count + len(reviews[i].split(" ")) < 1024:
      #print(cleaned_reviews[i])
      input = input + reviews[i] + " . "
      #print(input)
      count = count + len(reviews[i].split(" "))
    else:
      batches.append(input)
      #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
      #print("Number of reviews processed: {}".format(i))
      input = "summarize: " + reviews[i] + " . "
      count = len(reviews[i].split(" "))

  if(count != 0):
    batches.append(input)
    #print('Input {} created with a length of {} tokens.'.format(len(batches), count))
    #print("Number of reviews processed: {}".format(i))

  return batches

In [ ]:
def summarize_final(df):
  ids = df['product_id'].unique()
  years = df['review_year'].unique()
  for id in ids:
    for year in years:
      part = df[(df["product_id"] == id) & (df["review_year"] == year)]
      reviews = part["review_body"].tolist()
      batches = get_batches(reviews, False)

      summaries_first_stage = []
      for input in batches:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        # print(decoded_output)
        summaries_first_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      input = "summarize: "
      count = 0
      checkpoint = 0
      batches_second_stage = []
      i = 0
      for i in range(0,len(summaries_first_stage)):
        if count + len(summaries_first_stage[i].split(" ")) < 1024:
          #print(cleaned_reviews[i])
          input = input + summaries_first_stage[i] + " . "
          #print(input)
          count = count + len(summaries_first_stage[i].split(" "))
        else:
          batches_second_stage.append(input)
          #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
          #print("Number of reviews processed: {}".format(i))
          input = "summarize: " + summaries_first_stage[i] + " . "
          count = len(summaries_first_stage[i].split(" "))

      if(count != 0):
        batches_second_stage.append(input)
        #print('Input {} created with a length of {} tokens.'.format(len(batches_second_stage), count))
        #print("Number of reviews processed: {}".format(i))

      summaries_second_stage = []
      for input in batches_second_stage:
        inputs  = tokenizer.encode(input, return_tensors='tf', truncation=True, max_length=1024)
        output = model.generate(inputs, max_length=100)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        summaries_second_stage.append(nltk.sent_tokenize(decoded_output.strip())[0])

      print("Product " + str(id) + ", year " + str(year) + ": " + str(summaries_second_stage))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-xsum')
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-xsum')

In [ ]:
summaries =summarize_final(df)

In [ ]:
ds = df.groupby(by=['product_id', 'review_year'], sort=True).mean()
ds = ds.sort_values(by=['product_id', 'review_year'], ascending=[True, True])
ds = ds['star_rating'].tolist()
ds

In [ ]:
pipe = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
i = 0
for sum in summaries:
  val = pipe(str(sum))
  print("Second Stage: " + str(val[0].get("score")))
  print("Expected Rating: " + str(ds[i]/5))